5.1 Layers and Blocks

In [ ]:
!pip install d2l==0.14.2

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0324,  0.0366,  0.0455,  0.0510, -0.1234,  0.0992, -0.0431, -0.2923,
         -0.0099,  0.0463],
        [-0.1248,  0.0868,  0.0043, -0.0632, -0.0033,  0.2240, -0.1476, -0.1299,
          0.0059,  0.0527]], grad_fn=<AddmmBackward0>)

5.1.1 A Custom Block

In [3]:
class MLP(nn.Module):
  # Declare a layer with model parameters. Here, we declare two fully
  # connected layers
  def __init__(self):
    # Call the constructor of the `MLP` parent class `Module` to perform
    # the necessary initialization. In this way, other function arguments
    # can also be specified during class instantiation, such as the model
    # parameters, `params` (to be described later)
    super().__init__()
    self.hidden = nn.Linear(20, 256) # Hidden layer
    self.out = nn.Linear(256, 10) # Output layer

  # Define the forward propagation of the model, that is, how to return the
  # required model output based on the input `X`
  def forward(self, X):
    # Note here we use the funtional version of ReLU defined in the
    # nn.functional module.
    return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[ 0.1279, -0.0442,  0.0988, -0.0493,  0.1568,  0.2254, -0.0396,  0.0197,
          0.1076,  0.0400],
        [ 0.0608, -0.2831, -0.0094, -0.0964,  0.1772,  0.2115, -0.2277,  0.0780,
          0.2107,  0.0800]], grad_fn=<AddmmBackward0>)

5.1.2 The Sequential Block

In [5]:
class MySequential(nn.Module):
  def __init__(self, *args):
    super().__init__()
    for idx, module in enumerate(args):
      # Here, `module` is an instance of a `Module` subclass. We save it
      # in the member variable `_modules` of the `Module` class, and its
      # type is OrderedDict
      self._modules[str(idx)] = module
  
  def forward(self, X):
    # OrderedDict guarantees that members will be traversed in the order
    # they were added
    for block in self._modules.values():
      X = block(X)
    return X

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0455,  0.1705, -0.0056,  0.1519, -0.0145, -0.0107, -0.0912, -0.1370,
          0.0791,  0.0062],
        [ 0.1026,  0.2735,  0.2171,  0.0908, -0.1186,  0.0065, -0.1321, -0.3267,
          0.0666, -0.0111]], grad_fn=<AddmmBackward0>)

5.1.3 Executing Code in the Forward Propagation Function

In [7]:
class FixedHiddenMLP(nn.Module):
  def __init__(self):
      super().__init__()
      # Random weight parameters that will not compute gradients and
      # therefore keep constant during training
      self.rand_weight = torch.rand((20, 20), requires_grad=False)
      self.linear = nn.Linear(20, 20)
  
  def forward(self, X):
      X = self.linear(X)
      # Use the created constant parameters, as well as the `relu` and `mm`
      # functions
      X = F.relu(torch.mm(X, self.rand_weight) + 1)
      # Reuse the fully-connected layer. This is equivalent to sharing
      # parameters with two fully-connected layers
      X = self.linear(X)
      # Control flow
      while X.abs().sum() > 1:
          X /= 2
      return X.sum()

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(0.0673, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                  nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0031, grad_fn=<SumBackward0>)

5.2  Parameter Management

In [10]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.3267],
        [0.2170]], grad_fn=<AddmmBackward0>)

5.2.1 Parameter Access

In [11]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2207, -0.0208,  0.2451, -0.3311,  0.0021, -0.0247,  0.1263,  0.0284]])), ('bias', tensor([0.3219]))])


In [12]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.3219], requires_grad=True)
tensor([0.3219])


In [13]:
net[2].weight.grad == None

True

In [14]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [15]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                        nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # Nested here
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.2058],
        [-0.2058]], grad_fn=<AddmmBackward0>)

In [17]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [18]:
rgnet[0][1][0].bias.data

tensor([ 0.3192, -0.1652,  0.3070,  0.1421, -0.2780,  0.2812,  0.0919,  0.4930])

5.2.2 Parameter Initialization

In [19]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0069,  0.0031,  0.0075, -0.0101]), tensor(0.))

In [20]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [21]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.3210, -0.0780, -0.3346, -0.0604])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [22]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  0.0000, -6.0131,  0.0000],
        [-5.1322, -8.6761,  5.0347,  7.3095]], grad_fn=<SliceBackward0>)

In [23]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, -5.0131,  1.0000])

5.2.3 Tied Parameters

In [24]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


5.3 Custom Layers

5.3.1 Layers without Parameters

In [25]:
import torch
from torch import nn
from torch.nn import functional as F

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X - X.mean()

In [26]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [27]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [28]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(2.7940e-09, grad_fn=<MeanBackward0>)

5.3.2 Layers with Parameters

In [30]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [31]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.7731, -0.3101,  0.8888],
        [-0.3870,  0.6647,  0.9168],
        [-1.7930,  0.0925, -1.4515],
        [ 0.3857, -0.6245, -0.0789],
        [ 0.6853,  1.3649,  0.4122]], requires_grad=True)

In [32]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.2531, 0.0000],
        [0.9586, 0.3571, 0.0000]])

In [33]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[1.2949],
        [0.0000]])

5.4 File I/O

5.4.1 Loading and Saving Tensors

In [34]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [35]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [36]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [37]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

5.4.2 Loading and Saving Model Parameters

In [38]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [39]:
torch.save(net.state_dict(), 'mlp.params')

In [40]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [41]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

5.5 GPUs

In [ ]:
!nvidia-smi

In [43]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [44]:
torch.cuda.device_count()

0

In [45]:
def try_gpu(i=0): #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus(): #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    devices = [torch.device(f'cuda:{i}')
              for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

5.5.2 Tensors and GPUs

In [46]:
x = torch.tensor([1,2,3])
x.device

device(type='cpu')

In [47]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [48]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.8812, 0.8429, 0.9612],
        [0.1952, 0.7555, 0.0264]])

In [49]:
Z = X.cuda(1)
print(X)
print(Z)

RuntimeError: ignored

In [ ]:
Y + Z

In [ ]:
Z.cuda(1) is Z

5.5.3 Neural Networks and GPUs


In [52]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [53]:
net(X)

tensor([[0.6341],
        [0.6341]], grad_fn=<AddmmBackward0>)

In [54]:
net[0].weight.data.device

device(type='cpu')